In [1]:
import cutlass
import cutlass.cute as cute
from categories import *
from test_utils import *
from layout_utils import *

# Layout Categories

## Tuple morphisms

A tuple morphism $f:S \to T$ is encoded as an object of the class Tuple_morphism. For example:

In [27]:
f = Tuple_morphism(domain   = (256,256,512,512),
                   codomain = (256,2,512,4,512),
                   map      = (1,0,5,3))
print("f =",f)

f = (256, 256, 512, 512):(256, 2, 512, 4, 512):(1, 0, 5, 3)


If $f$ is a tuple morphism, then we can compute the associated flat layout $L_f$:

In [28]:
@cute.jit
def example():
    f = Tuple_morphism(domain   = (256,256,512,512),
                       codomain = (256,2,512,4,512),
                       map      = (1,0,5,3))
    L_f = compute_flat_layout(f)
    print("L_f = ",L_f)
example()

L_f =  (256,256,512,512):(1,0,1048576,512)


If $L$ is a flat layout, we can check if $L$ is tractable, and if so, produce a tuple morphism $f$ with $L_f = L$.

In [29]:
@cute.jit
def example():
    L = cute.make_layout((2,2,2),stride = (8,2,512))
    f = compute_Tuple_morphism(L)
    print(f"{'L':<4}= {L}")
    print(f"{'L_f':<4}= {compute_flat_layout(f)}")
example()

L   = (2,2,2):(8,2,512)
L_f = (2,2,2):(8,2,512)


## Operations on Tuple morphisms

### Composition

If $f:S \to T$ and $g:T \to U$ are tuple morphisms, then we can compose $f$ and $g$ to form the tuple morphism $g \circ f: S \to U$:

In [32]:
f = Tuple_morphism((2,2),(10,2,10,2),(2,4))
g = Tuple_morphism((10,2,10,2),(10,10,2,2),(1,3,2,4))
composite = f.compose(g)
composite.name = "g o f"
print("g o f =",composite)

g o f = (2, 2):(10, 10, 2, 2):(3, 4)


Composition of tuple morphisms is compatible with composition of layouts, in that

$L_{g \circ f} = L_g \circ L_f$:

In [9]:
@cute.jit
def example():
    f = Tuple_morphism((8,8,8,2),(8,2,8,4,8),(3,1,5,2))
    g = Tuple_morphism((8,2,8,4,8),(8,8,2,2),(1,3,2,0,0))
    composite = f.compose(g)
    layout_composite = compute_flat_layout(composite)
    composite_layout = cute.composition(compute_flat_layout(g),compute_flat_layout(f))
    print(f"{'L_{g o f}':<10}= {layout_composite}")
    print(f"{'L_g o L_f':<10}= {composite_layout}")
example()

L_{g o f} = (8,8,8,2):(8,1,0,64)
L_g o L_f = (8,8,8,2):(8,1,0,64)


### Sort

We can sort tuple morphisms:

In [37]:
f = Tuple_morphism((16,8,8,32,256),(2,2,8,16,8,4,32),(4,3,5,7,0),name = "f")
f_sorted = f.sort()
print(f"{'f':<7} = {f}")
print(f"{'sort(f)':<7} = {f_sorted}")

f       = (16, 8, 8, 32, 256):(2, 2, 8, 16, 8, 4, 32):(4, 3, 5, 7, 0)
sort(f) = (256, 8, 16, 8, 32):(2, 2, 8, 16, 8, 4, 32):(0, 3, 4, 5, 7)


### Coalesce

We can coalesce tuple morphisms:

In [38]:
f = Tuple_morphism((2,3,2,3,10),(4,2,3,3,2,3,10),(2,3,5,6,7))
f_coalesced = f.coalesce()
print(f"{'f':<11} = {f}")
print(f"{'coalesce(f)':<11} = {f_coalesced}")

f           = (2, 3, 2, 3, 10):(4, 2, 3, 3, 2, 3, 10):(2, 3, 5, 6, 7)
coalesce(f) = (6, 60):(4, 6, 3, 60):(2, 4)


This operation is compatible with the coalesce operation on layouts, in that $L_{\text{coalesce}(f)} = \text{coalesce}(L_f)$.

In [39]:
@cute.jit
def example():
    f             = Tuple_morphism((2,2,3,3,5,5),(4,2,2,3,3,10,5,5),(2,3,5,4,7,8),name = "f")
    f_coalesced   = f.coalesce()
    L_f           = compute_flat_layout(f)
    L_f_coalesced = compute_flat_layout(f_coalesced)
    print(f"{'L_coalesce(f)':<12} = {L_f_coalesced}")
    print(f"{'coalesce(L_f)':<12} = {cute.coalesce(L_f)}")
example()

L_coalesce(f) = (4,3,3,25):(4,48,16,1440)
coalesce(L_f) = (4,3,3,25):(4,48,16,1440)


### Concatenate

We can concatenate tuple morphisms $f$ and $g$ with the same codomain and disjoint images.

In [45]:
f = Tuple_morphism((10,10),(5,10,5,10,5,10),(6,4))
g = Tuple_morphism((5,5,5),(5,10,5,10,5,10),(3,1,0))
concatenated_morphism = f.concat(g)
print(f"{'f':<11}={f}")
print(f"{'g':<11}={g}")
print(f"{'concat(f,g)'}={concatenated_morphism}")

f          =(10, 10):(5, 10, 5, 10, 5, 10):(6, 4)
g          =(5, 5, 5):(5, 10, 5, 10, 5, 10):(3, 1, 0)
concat(f,g)=(10, 10, 5, 5, 5):(5, 10, 5, 10, 5, 10):(6, 4, 3, 1, 0)


### Complement

We can form the complement $f^*$ of a tuple morphism $f$:

In [49]:
f = Tuple_morphism((5,5),(2,5,4,5),(2,4))
f_complement = f.complement()
print(f"{'f':<13}={f}")
print(f"{'complement(f)'}={f_complement}")


f            =(5, 5):(2, 5, 4, 5):(2, 4)
complement(f)=(2, 4):(2, 5, 4, 5):(1, 3)
